In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import time

In [ ]:
start = time.time()

# 크롬 띄움
driver = webdriver.Chrome()

# 장르별 페이지 (판타지, 현대판타지, 무협, 드라마, 로맨스판타지, 로맨스)
targetSites = ['https://page.kakao.com/menu/10011/screen/91', 'https://page.kakao.com/menu/10011/screen/64', 'https://page.kakao.com/menu/10011/screen/70', 
               'https://page.kakao.com/menu/10011/screen/100', 'https://page.kakao.com/menu/10011/screen/92', 'https://page.kakao.com/menu/10011/screen/68']

# 장르별 top300 데이터 저장할 리스트 초기화
Genres = [] # 장르 저장할 리스트
Titles = [] # 제목 저장할 리스트
Author = [] # 작가 저장할 리스트
Count = []  # 연재수 연재수
View = [] # 전체 조회수
Start = []  # 연재 시작일
Last = []  # 마지막 연재일
Point = [] # 평점 리스트
Comment = [] # 댓글 수
n=0
for targetSite in targetSites:
    driver.get(targetSite)
    wait = WebDriverWait(driver, 10)
    
    # 장르별 하이퍼 링크 저장할 리스트 초기화
    Links=[] 
    
    for i in range(6): # 가장 마지막 까지 스크롤
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(0.5)

    # XPATH후 클릭이 안되서 아래까지 스크롤 한 후 하이퍼링크만 가져와서 저장함
    for i in range(1, 301, 1):
        time.sleep(0.1)
        try:
            element = f'#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div > div.flex.grow.flex-col > div:nth-child(5) > div > div.flex.w-full.grow.flex-col > div > div > div > div:nth-child({i}) > div > a'
            div=driver.find_element(By.CSS_SELECTOR, element).get_attribute('href')
            Links.append(div)
        except:
            break
    
    if targetSite == 'https://page.kakao.com/menu/10011/screen/91':
        Genre = '판타지'
    elif targetSite == 'https://page.kakao.com/menu/10011/screen/64':
        Genre = '현대판타지'
    elif targetSite == 'https://page.kakao.com/menu/10011/screen/70':
        Genre = '무협'
    elif targetSite == 'https://page.kakao.com/menu/10011/screen/100':
        Genre = '드라마'
    elif targetSite == 'https://page.kakao.com/menu/10011/screen/92':
        Genre = '로맨스판타지'
    else:
        Genre = '로맨스'    
    
    #print(Genre)
    #print(type(Genre))
    #print(len(Links))
    
    for i in range(len(Links)):
        driver.get(Links[i])

        try:
            # 연재 시작일
            startdate = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/div/div/a/div/div[2]/div[2]/span'
            wait.until(EC.presence_of_element_located((By.XPATH, startdate)))
            Start.append(driver.find_element(By.XPATH, startdate).text)
            # 장르
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[1]/div')))
            Genres.append(Genre)
            # 제목
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[1]')))
            Titles.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[1]').text)
            # 작가
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[2]')))
            Author.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/span[2]').text)
            # 평점
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[3]/span')))
            Point.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[3]/span').text)
            # 조회수
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[2]/span')))
            View.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[1]/div[1]/div/div[2]/a/div/div[1]/div[2]/span').text)
            # 댓글 수
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[3]/div/div[1]/div[1]/span')))
            Comment.append(driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[3]/div/div[1]/div[1]/span').text)
            
            # 연재수 
            countselect = '#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1 > div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > div.flex-1.flex.flex-col > div.rounded-b-12pxr.bg-bg-a-20 > div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.bg-bg-a-20.px-18pxr > div.flex.h-full.flex-1.items-center.space-x-8pxr > span'
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, countselect)))
            Count.append(driver.find_element(By.CSS_SELECTOR, countselect).text)
            
            # 최근 연재일
            Sort=driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div/span').click()
            SortDESC = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[1]/div[3]/div[2]/div').click()
            time.sleep(0.5)
            lastdate = '//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/ul/li[1]/div/div/a/div/div[2]/div[2]/span'
            Last.append(driver.find_element(By.XPATH, lastdate).text)
            
            print(f'{Genre}의 {i+1}번, 제목: {Titles[n]}, 작가: {Author[n]}, 평점: {Point[n]}, 연재수: {Count[n]}, 조회수: {View[n]}, 댓글수: {Comment[n]}, 연재시작일: {Start[n]}, 최근 업데이트: {Last[n]}')
            n += 1 
            
        except (NoSuchElementException, TimeoutException): # 여기선 네이버랑 다르게 에러가 안떠서 간단하게 함
            driver.back()
            Genres.append('19금')
            Titles.append('19금')
            Author.append('19금')
            Count.append('19금') 
            Point.append('19금')
            View.append('19금')
            Comment.append('19금')
            Start.append('19금')
            Last.append('19금')
            time.sleep(0.5)
            print(f'{Genre}의 {i+1}번, 19금입니다')
            n += 1
            continue 
    
end = time.time()
print(end-start)
    
# 데이터프레임으로 만듬
df = pd.DataFrame({'제목': Titles, '장르': Genres, '작가':Author,'조회수':View, '댓글': Comment, '연재수': Count, '연재 시작일': Start, '최신 연재일': Last,'평점': Point})
df.index = df.index + 1  
df